Import all variables/ libraries from main

In [2]:
from main import data, pd, plt, train_test_split, GaussianNB, accuracy_score, DecisionTreeClassifier, export_text, plot_tree, sns, apriori, association_rules
import constants
import utils

Print data

In [ ]:
data

Plot data, shows how often MonkeyPox is negative and positive

In [ ]:
series = data[constants.MONKEY_POX].value_counts()
#series2,
# TODO: Try to use countplot and change colors to normal blue and orange
plt.bar(series.index, series.values, color=constants.COLOR_PALETTE)
plt.title('Cases of Monkey Pox seperated by positive and negative')
plt.show()

Plots heatmap: Shows correlations between the different features

In [8]:
sns.heatmap(data.corr(), annot=True, vmin=0.5, vmax=0.7)
plt.show()

/tmp/ipykernel_15947/3714203276.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  sns.heatmap(data.corr(), annot=True, vmin=0.5, vmax=0.7)


Shows how often each systemic illness is causing MonkeyPox to be positive or negative

In [7]:
# lift = 1
# confidence = 1 is good?
symptoms_df = utils.clean_data(data)
frequency = apriori(symptoms_df, min_support=0.005, use_colnames=True)
rules = association_rules(frequency, metric='lift')
rules.sort_values('confidence', ascending=False, inplace=True)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
6368,"(Solitary_Lesion, Rectal_Pain, Sore_Throat, Sw...",(Systemic_Illness),0.039412,1.000000,0.039412,1.000000,1.000000,0.000000,inf
7609,"(Oral_Lesions, Rectal_Pain, Swollen_Tonsils, P...",(Systemic_Illness),0.037903,1.000000,0.037903,1.000000,1.000000,0.000000,inf
1208,"(Penile_Oedema, HIV_Infection, STI)",(Systemic_Illness),0.153372,1.000000,0.153372,1.000000,1.000000,0.000000,inf
7485,"(Oral_Lesions, Solitary_Lesion, Rectal_Pain, P...",(Systemic_Illness),0.039035,1.000000,0.039035,1.000000,1.000000,0.000000,inf
7546,"(Oral_Lesions, Rectal_Pain, Swollen_Tonsils, H...",(Systemic_Illness),0.041612,1.000000,0.041612,1.000000,1.000000,0.000000,inf
...,...,...,...,...,...,...,...,...,...
18646,"(Systemic_Illness, HIV_Infection)","(Oral_Lesions, Solitary_Lesion, Rectal_Pain, S...",0.558615,0.009931,0.005406,0.009677,0.974382,-0.000142,0.999743
18147,(HIV_Infection),"(Oral_Lesions, Solitary_Lesion, Rectal_Pain, S...",0.558615,0.009931,0.005406,0.009677,0.974382,-0.000142,0.999743
18656,(HIV_Infection),"(Oral_Lesions, Solitary_Lesion, Rectal_Pain, S...",0.558615,0.009931,0.005406,0.009677,0.974382,-0.000142,0.999743
17640,(Systemic_Illness),"(Oral_Lesions, Solitary_Lesion, Rectal_Pain, S...",1.000000,0.009617,0.009617,0.009617,1.000000,0.000000,1.000000


In [6]:
# TODO: Change legend and axises
sns.countplot(x=data[constants.SYSTEMIC_ILLNESS], hue=data[constants.MONKEY_POX], palette=constants.COLOR_PALETTE)
plt.show()

Plots data, using pairplot

In [ ]:
#sns.pairplot(data.head(100), hue='MonkeyPox')

Calculates accuracy, using GaussianNB()

In [ ]:
simple_dict = {'None': 1, 'Fever': 2, 'Swollen Lymph Nodes': 3, 'Muscle Aches and Pain': 4}
data[constants.SYSTEMIC_ILLNESS] = [simple_dict[item] for item in data[constants.SYSTEMIC_ILLNESS]]
data_tree = data.iloc[:,:-1]
target = data.iloc[:,-1:]
ran_stream = 23
x_train, x_test, y_train, y_test = train_test_split(data_tree, target, random_state=ran_stream)

model = GaussianNB()
model.fit(x_train, y_train)
y_prediction = model.predict(x_test)
print(100 * accuracy_score(y_test, y_prediction))

Creates DecisionTree, by using entropy

In [ ]:
model_tree = DecisionTreeClassifier(criterion='entropy', splitter='best', min_samples_split=5)
model_tree.fit(data_tree, target)
text_representation = export_text(model_tree, feature_names=data_tree.columns.values.tolist())
figure = plt.figure(figsize=(10,8))
plot_tree(model_tree, feature_names=data_tree.columns, filled=True, rounded=True)
plt.show()


Plots data, when symptom (2-7) is True and MonkeyPox is Positive

In [ ]:
symptoms = data.iloc[:, 1:-1].columns.values.tolist()
symptoms_df = data.iloc[:, 1:]
symptoms_df = utils.fetch_true_symptoms(symptoms, symptoms_df)
print(symptoms_df)

pos_lst, negative_lst = utils.create_outcome_lists(symptoms, symptoms_df)
df1 = pd.DataFrame({'Symptoms': symptoms, 'Number of infections': pos_lst})
df2 = pd.DataFrame({'Symptoms': symptoms, 'Number of infections': negative_lst})
df1['outcome'] = 'Positive'
df2['outcome'] = 'Negative'
res = pd.concat([df1, df2])
sns.barplot(x='Symptoms', y='Number of infections', data=res, hue='outcome', palette=constants.COLOR_PALETTE)
plt.show()

Calculates accuracy by removing different symptoms, checks if accuracy is higher or lower depending on which symptom is removed

In [9]:
simple_dict = {'None': 1, 'Fever': 2, 'Swollen Lymph Nodes': 3, 'Muscle Aches and Pain': 4}
data[constants.SYSTEMIC_ILLNESS] = [simple_dict[item] for item in data[constants.SYSTEMIC_ILLNESS]]
data_tree = data.iloc[:,:-1]
target = data.iloc[:,-1:]
utils.create_feature_accuracy_dict(data_tree, target, data.iloc[:, 1:-1].columns.values.tolist())

{"['Systemic_Illness', 'Sore_Throat', 'Penile_Oedema', 'Oral_Lesions', 'Solitary_Lesion', 'Swollen_Tonsils', 'HIV_Infection', 'STI']": 66.4,
 "['Systemic_Illness', 'Rectal_Pain', 'Penile_Oedema', 'Oral_Lesions', 'Solitary_Lesion', 'Swollen_Tonsils', 'HIV_Infection', 'STI']": 67.67999999999999,
 "['Systemic_Illness', 'Rectal_Pain', 'Sore_Throat', 'Oral_Lesions', 'Solitary_Lesion', 'Swollen_Tonsils', 'HIV_Infection', 'STI']": 67.75999999999999,
 "['Systemic_Illness', 'Rectal_Pain', 'Sore_Throat', 'Penile_Oedema', 'Solitary_Lesion', 'Swollen_Tonsils', 'HIV_Infection', 'STI']": 67.648,
 "['Systemic_Illness', 'Rectal_Pain', 'Sore_Throat', 'Penile_Oedema', 'Oral_Lesions', 'Swollen_Tonsils', 'HIV_Infection', 'STI']": 67.75999999999999,
 "['Systemic_Illness', 'Rectal_Pain', 'Sore_Throat', 'Penile_Oedema', 'Oral_Lesions', 'Solitary_Lesion', 'HIV_Infection', 'STI']": 68.24,
 "['Systemic_Illness', 'Rectal_Pain', 'Sore_Throat', 'Penile_Oedema', 'Oral_Lesions', 'Solitary_Lesion', 'Swollen_Tonsils',